In [0]:
def exec_spark_sql(query,config_data=None):
    try:
        df = spark.sql(query)
        return df
    except Exception as e:
        raise Exception(f'exception in exec_spark_sql: {e} : query: {query}')

In [0]:
# pre and post script auditing status description
def get_status_desc(status_num):
    status_desc = {
        1: "'STARTED'",
        2: "'ERROR_ENCOUNTERED'",
        3: "'COMPLETED'",
        4: "'ACTIVE'",
        5: "'INACTIVE'"
    } 
    return status_desc.get(status_num, "'nothing'")

In [0]:
# auditing for audit_log and control_table
def init_ssi_audit(config_data,table,job_num,batch_start_date,batch_end_date,src_count,target_count,buss_start_dt,batch_sub_batch_id,status_num,failure_point="null",technical_error_code="null",technical_error_desc="null"):
    try:
        log.info(f"buss_start_dt: {buss_start_dt}")
#         log.info(f"buss_start_dt: {buss_start_dt}")
        buss_end_dt_query = f"select max({table.get('incr_indentifier_cols')}) from {table.get('wf_name')}"
        buss_end_dt_df = get_df(config_data,buss_end_dt_query)
        buss_end_dt = buss_end_dt_df.collect()[0][0]
        log.info(f"buss_end_dt: {buss_end_dt}")
        audit_log_query = f"insert into {config_data.get('source').get('audit_schema_nm')}.audit_log values(1,{job_num},{batch_sub_batch_id.get('batch_id')},{batch_sub_batch_id.get('sub_batch_id')},{table.get('wf_num')},'{table.get('wf_name')}','{batch_start_date}','{batch_end_date}','{buss_start_dt}','{buss_end_dt}',{status_num},null,{failure_point},{technical_error_code},{get_status_desc(status_num)},{technical_error_desc},null,null)"
        log.debug(f"audit_log_query: {audit_log_query}")
        
        match_flag = 1 if src_count == target_count else 0
        control_table_query = f"insert into {config_data.get('source').get('audit_schema_nm')}.control_table values({job_num},{batch_sub_batch_id.get('batch_id')},{batch_sub_batch_id.get('sub_batch_id')},{table.get('wf_num')},'{table.get('wf_name')}',null,{src_count},{target_count},null,{match_flag},null,null,null,null,null,null)"
        log.debug(f"control_table_query: {control_table_query}")
        
        exec_spark_sql(audit_log_query)
        exec_spark_sql(control_table_query)
    except Exception as e:
        raise Exception('exception in init_ssi_audit: %s' % e)